In [26]:
import pandas as pd
import numpy as np
import os

In [3]:
current_directory = os.getcwd()

# Navigate back two folders
target_directory = os.path.abspath(os.path.join(current_directory, '..', 'Data', 'PPT-Ohmnet'))

# Navigate to folder A, B, and then access file C.edgelist
file_path = os.path.join(target_directory, 'G-SynMiner_miner-geneHUGO.tsv')


In [6]:
try:
    genes_DF = pd.read_csv(file_path, sep = "	", index_col = 0)
    print("CSV file successfully read.")  # Indicate successful reading
except FileNotFoundError:
    print("File not found. Please check the file path.")  # Handle file not found error

# Now you can use 'df' DataFrame for further analysis

CSV file successfully read.


C:\Users\inigo\AppData\Local\Temp\ipykernel_32840\2566101220.py:2: DtypeWarning: Columns (32,34,38) have mixed types. Specify dtype option on import or set low_memory=False.
  genes_DF = pd.read_csv(file_path, sep = "	", index_col = 0)


In [7]:
genes_DF

,hgnc_id,symbol,name,locus_group,locus_type,status,location,location_sortable,alias_symbol,alias_name,...,horde_id,merops,imgt,iuphar,kznf_gene_catalog,mamit-trnadb,cd,lncrnadb,enzyme_id,intermediate_filament_db
# ensembl_gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000121410,HGNC:5,A1BG,alpha-1-B glycoprotein,protein-coding gene,gene with protein product,Approved,19q13.43,19q13.43,NaN,NaN,...,NaN,I43.950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000268895,HGNC:37133,A1BG-AS1,A1BG antisense RNA 1,non-coding RNA,"RNA, long non-coding",Approved,19q13.43,19q13.43,FLJ23569,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000148584,HGNC:24086,A1CF,APOBEC1 complementation factor,protein-coding gene,gene with protein product,Approved,10q21.1,10q21.1,ACF|ASP|ACF64|ACF65|APOBEC1CF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000175899,HGNC:7,A2M,alpha-2-macroglobulin,protein-coding gene,gene with protein product,Approved,12p13.31,12p13.31,FWP007|S863-7|CPAMD5,NaN,...,NaN,I39.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000245105,HGNC:27057,A2M-AS1,A2M antisense RNA 1 (head to head),non-coding RNA,"RNA, long non-coding",Approved,12p13.31,12p13.31,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000162378,HGNC:25820,ZYG11B,"zyg-11 family member B, cell cycle regulator",protein-coding gene,gene with protein product,Approved,1p32.3,01p32.3,FLJ13456,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000159840,HGNC:13200,ZYX,zyxin,protein-coding gene,gene with protein product,Approved,7q32,07q32,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000274572,HGNC:51695,ZYXP1,zyxin pseudogene 1,pseudogene,pseudogene,Approved,8q24.23,08q24.23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
current_directory = os.getcwd()

# Navigate back two folders
target_directory = os.path.abspath(os.path.join(current_directory, '..', 'Data', 'PPT-Ohmnet'))

# Navigate to folder A, B, and then access file C.edgelist
file_path = os.path.join(target_directory, 'PPT-Ohmnet_tissues-combined.edgelist')


In [18]:
try:
    edgelist_DF = pd.read_csv(file_path, sep = "	")
    print("CSV file successfully read.")  # Indicate successful reading
except FileNotFoundError:
    print("File not found. Please check the file path.")  # Handle file not found error

# Now you can use 'df' DataFrame for further analysis

CSV file successfully read.


In [19]:
edgelist_DF

,protein1,protein2,tissue
0,4790,79155,urinary_bladder
1,26039,6597,urinary_bladder
2,57154,3309,urinary_bladder
3,6631,9775,urinary_bladder
4,1855,8312,urinary_bladder
...,...,...,...
3666558,6045,1153,spermatocyte
3666559,10616,1387,spermatocyte
3666560,2033,7026,spermatocyte
3666561,324,1457,spermatocyte


In [58]:
vals = list(edgelist_DF['protein1'].values)
vals2 = list(edgelist_DF['protein2'].values)
genes = list(set(vals + vals2))

In [59]:
genes

[2,
 8200,
 81929,
 8202,
 81930,
 8204,
 6515,
 14,
 16,
 19,
 20,
 23,
 8218,
 27,
 253980,
 29,
 31,
 32,
 34,
 57379,
 37,
 147495,
 8233,
 41,
 43,
 8237,
 8239,
 48,
 50,
 8243,
 204851,
 51,
 57396,
 54,
 58,
 57403,
 57402,
 59,
 57405,
 8260,
 70,
 71,
 8266,
 8273,
 81,
 114771,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 8289,
 8291,
 100,
 102,
 8295,
 57447,
 57448,
 8294,
 107,
 131177,
 8301,
 114798,
 111,
 114799,
 113,
 57449,
 114803,
 90226,
 114805,
 118,
 119,
 8312,
 8313,
 8314,
 8315,
 123,
 127,
 128,
 8321,
 8322,
 8323,
 8324,
 133,
 134,
 135,
 136,
 8325,
 130,
 131,
 114822,
 142,
 146,
 57491,
 148,
 147,
 150,
 151,
 57496,
 153,
 154,
 155,
 156,
 152,
 158,
 57502,
 57506,
 57508,
 166,
 8359,
 8360,
 8361,
 8362,
 8363,
 8364,
 8365,
 8366,
 8367,
 8368,
 57521,
 8370,
 57514,
 177,
 181,
 182,
 90293,
 57522,
 185,
 186,
 183,
 189,
 57534,
 190,
 196,
 197,
 199,
 8394,
 8395,
 8396,
 8398,
 207,
 208,
 57551,
 210,
 8399,
 57555,
 212,
 214,
 84

In [60]:
geneList = genes_DF['hgnc_id'].values

# Remove the 'hgnc:' part from each item in the list
modified_list = [int(item.split(':')[1]) for item in geneList]
modified_list

[5,
 37133,
 24086,
 7,
 27057,
 23336,
 41022,
 41523,
 8,
 30005,
 18149,
 17968,
 13666,
 21298,
 18226,
 17,
 24427,
 50301,
 32037,
 32038,
 50305,
 17929,
 16881,
 25662,
 19679,
 30205,
 18,
 19,
 15886,
 33842,
 20,
 21022,
 28417,
 49894,
 23993,
 14235,
 17366,
 51526,
 19235,
 21,
 40053,
 49667,
 23,
 29,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 39983,
 30,
 31,
 14637,
 14638,
 32972,
 40,
 45,
 46,
 47,
 48,
 49,
 50,
 41,
 14114,
 31129,
 31130,
 42,
 51,
 53,
 54,
 55,
 56,
 40055,
 57,
 33352,
 33353,
 59,
 60,
 52,
 14639,
 14640,
 16022,
 61,
 63,
 64,
 65,
 42662,
 66,
 67,
 68,
 69,
 70,
 71,
 48841,
 48842,
 72,
 73,
 74,
 13884,
 13886,
 13887,
 17553,
 18717,
 18718,
 20154,
 21396,
 21398,
 23759,
 25656,
 16407,
 18289,
 15868,
 19837,
 20293,
 24538,
 38856,
 28235,
 26971,
 49685,
 13921,
 16128,
 28756,
 28394,
 34041,
 34042,
 34043,
 34044,
 38508,
 38509,
 38510,
 24278,
 26925,
 26111,
 11320,
 24011,
 29859,
 17265,
 76,
 77,
 78,
 19195,
 29132,
 79,


In [62]:
found_genes = []
for i in modified_list:
    if i in genes:
        found_genes += [i]
print("genes in graph: " + str(len(found_genes)) + "/" + str(len(modified_list)))

genes in graph: 2549/35654


In [63]:
found_genes

[19,
 20,
 23,
 29,
 32,
 34,
 37,
 31,
 48,
 50,
 41,
 51,
 54,
 59,
 70,
 71,
 23759,
 81,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 23338,
 4035,
 1611,
 118,
 119,
 123,
 127,
 3569,
 3570,
 10522,
 26060,
 130,
 134,
 135,
 142,
 150,
 146,
 147,
 148,
 154,
 155,
 51499,
 166,
 51621,
 177,
 186,
 210,
 212,
 214,
 216,
 189,
 190,
 196,
 197,
 199,
 207,
 208,
 220,
 223,
 226,
 23299,
 238,
 239,
 240,
 241,
 943,
 3337,
 249,
 22976,
 265,
 283,
 285,
 286,
 288,
 291,
 7135,
 317,
 23499,
 320,
 5175,
 51741,
 51747,
 321,
 324,
 328,
 329,
 330,
 333,
 336,
 338,
 341,
 344,
 348,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 367,
 373,
 374,
 369,
 382,
 384,
 387,
 388,
 389,
 391,
 392,
 395,
 396,
 399,
 402,
 409,
 405,
 403,
 406,
 408,
 23161,
 23162,
 429,
 430,
 463,
 467,
 468,
 473,
 476,
 477,
 478,
 487,
 491,
 23643,
 25970,
 1183,
 538,
 539,
 540,
 545,
 546,
 549,
 554,
 558,
 566,
 567,
 570,
 572,
 573,
 576,
 578,
 579,
 580,
 581,
 582,
 595,
 597,
 598,
 60